In [13]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

import time
import argparse
import os
import numpy as np

# Our Modules
import datasets
from datasets import utils
from models.MPNN import MPNN
from models.MPNN_Duvenaud import MpnnDuvenaud
from models.MPNN_GGNN import MpnnGGNN
from models.MPNN_IntNet import MpnnIntNet
from LogMetric import AverageMeter, Logger

from LogMetric import AverageMeter, Logger

def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

# Argument parser
parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='our_data')
parser.add_argument('--datasetPath', default='./data-parse/chemfindata/', help='dataset path')
parser.add_argument('--logPath', default='./log/our_data/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/our_data/mpnn/', help='plot path')
parser.add_argument('--resume', default='./checkpoint/our_data/mpnn/',
                    help='path to latest checkpoint')
# Optimization Options
parser.add_argument('--batch-size', type=int, default=100, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=360, metavar='N',
                    help='Number of epochs to train (default: 360)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=2, help='Pre-fetching threads.')

best_er1 = 0



args = parser.parse_args([])

args.cuda = not args.no_cuda and torch.cuda.is_available()

# Load data
root = args.datasetPath

print('Prepare files')
files = [f for f in os.listdir(root) if os.path.isfile(os.path.join(root, f))]

idx = np.random.permutation(len(files))
idx = idx.tolist()

valid_ids = [files[i] for i in idx[0:10000]]
test_ids = [files[i] for i in idx[10000:20000]]
train_ids = [files[i] for i in idx[20000:]]

data_train = datasets.Qm9(root, train_ids, edge_transform=utils.qm9_edges, e_representation='raw_distance')
data_valid = datasets.Qm9(root, valid_ids, edge_transform=utils.qm9_edges, e_representation='raw_distance')
data_test = datasets.Qm9(root, test_ids, edge_transform=utils.qm9_edges, e_representation='raw_distance')

In [14]:
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='our_data')
parser.add_argument('--datasetPath', default='./data-parse/chemfindata/', help='dataset path')
parser.add_argument('--logPath', default='./log/our_data/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/our_data/mpnn/', help='plot path')
parser.add_argument('--resume', default='./checkpoint/our_data/mpnn/',
                    help='path to latest checkpoint')
# Optimization Options
parser.add_argument('--batch-size', type=int, default=100, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=360, metavar='N',
                    help='Number of epochs to train (default: 360)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=2, help='Pre-fetching threads.')

best_er1 = 0
reader_folder = os.path.realpath( os.path.abspath('../GraphReader'))
if reader_folder not in sys.path:
    sys.path.insert(1, reader_folder)

In [12]:
def init_graph(prop):
    
    prop = prop.split()
    g_tag = prop[0]
    g_index = int(prop[1])
    g_A = float(prop[2])
    g_B = float(prop[3]) 
    g_C = float(prop[4]) 
    g_mu = float(prop[5])
    g_alpha = float(prop[6]) 
    g_homo = float(prop[7])
    g_lumo = float(prop[8]) 
    g_gap = float(prop[9])
    g_r2 = float(prop[10])
    g_zpve = float(prop[11]) 
    g_U0 = float(prop[12]) 
    g_U = float(prop[13])
    g_H = float(prop[14])
    g_G = float(prop[15])
    g_Cv = float(prop[16])

    labels = [g_mu, g_alpha, g_homo, g_lumo, g_gap, g_r2, g_zpve, g_U0, g_U, g_H, g_G, g_Cv]
    return nx.Graph(tag=g_tag, index=g_index, A=g_A, B=g_B, C=g_C, mu=g_mu, alpha=g_alpha, homo=g_homo,
                    lumo=g_lumo, gap=g_gap, r2=g_r2, zpve=g_zpve, U0=g_U0, U=g_U, H=g_H, G=g_G, Cv=g_Cv), labels

def init_graph_our(g_tag,prop):
    
    prop = prop.split()
    g_H=float(prop[0])
    g_p=float(prop[1])

    labels = [g_H, g_p]
    return nx.Graph(tag=g_tag,H=g_H, p=g_p), labels


# XYZ file reader for QM9 dataset
def xyz_graph_reader(graph_file):

    with open(graph_file,'r') as f:
        # Number of atoms
        #na = int(f.readline())
        tag=f.readline().strip()
        # Graph properties
        properties = f.readline()
        na=int(f.readline())
        #g, l = init_graph(properties)
        g,l=init_graph_our(tag, properties)
        atom_properties = []
        # Atoms properties
        for i in range(na):
            a_properties = f.readline()
            a_properties = a_properties.replace('.*^', 'e')
            a_properties = a_properties.replace('*^', 'e')
            a_properties = a_properties.split()
            atom_properties.append(a_properties)

        # Frequencies
        #f.readline()

        # SMILES
        smiles = f.readline()
        smiles = smiles.split()
        smiles = smiles[0]
        
        m = Chem.MolFromSmiles(smiles)
        m = Chem.AddHs(m)
        
        fdef_name = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
        factory = ChemicalFeatures.BuildFeatureFactory(fdef_name)
        feats = factory.GetFeaturesForMol(m)

        # Create nodes
        for i in range(0, m.GetNumAtoms()):
            atom_i = m.GetAtomWithIdx(i)

            g.add_node(i, a_type=atom_i.GetSymbol(), a_num=atom_i.GetAtomicNum(), acceptor=0, donor=0,
                       aromatic=atom_i.GetIsAromatic(), hybridization=atom_i.GetHybridization(),
                       num_h=atom_i.GetTotalNumHs(), coord=np.array(atom_properties[i][1:4]).astype(np.float),
                       pc=float(atom_properties[i][4]))

        for i in range(0, len(feats)):
            if feats[i].GetFamily() == 'Donor':
                node_list = feats[i].GetAtomIds()
                for i in node_list:
                    g.node[i]['donor'] = 1
            elif feats[i].GetFamily() == 'Acceptor':
                node_list = feats[i].GetAtomIds()
                for i in node_list:
                    g.node[i]['acceptor'] = 1

        # Read Edges
        for i in range(0, m.GetNumAtoms()):
            for j in range(0, m.GetNumAtoms()):
                e_ij = m.GetBondBetweenAtoms(i, j)
                if e_ij is not None:
                    g.add_edge(i, j, b_type=e_ij.GetBondType(),
                               distance=np.linalg.norm(g.node[i]['coord']-g.node[j]['coord']))
                else:
                    # Unbonded
                    g.add_edge(i, j, b_type=None,
                               distance=np.linalg.norm(g.node[i]['coord'] - g.node[j]['coord']))
    return g , l


In [4]:
from __future__ import print_function

import rdkit
import torch
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import numpy as np
import shutil
import os




def qm9_nodes(g, hydrogen=False):
    h = []
    for n, d in list(g.nodes(data=True)):
        h_t = []
        # Atom type (One-hot H, C, N, O F)
        h_t += [int(d['a_type'] == x) for x in ['H', 'C', 'N', 'O', 'F']]
        # Atomic number
        h_t.append(d['a_num'])
        # Partial Charge
        h_t.append(d['pc'])
        # Acceptor
        h_t.append(d['acceptor'])
        # Donor
        h_t.append(d['donor'])
        # Aromatic
        h_t.append(int(d['aromatic']))
        # Hybradization
        h_t += [int(d['hybridization'] == x) for x in [rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2, rdkit.Chem.rdchem.HybridizationType.SP3]]
        # If number hydrogen is used as a
        if hydrogen:
            h_t.append(d['num_h'])
        h.append(h_t)
    return h


def qm9_edges(g, e_representation='raw_distance'):
    remove_edges = []
    e={}    
    for n1, n2, d in g.edges_iter(data=True):
        e_t = []
        # Raw distance function
        if e_representation == 'chem_graph':
            if d['b_type'] is None:
                remove_edges += [(n1, n2)]
            else:
                e_t += [i+1 for i, x in enumerate([rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC])
                        if x == d['b_type']]
        elif e_representation == 'distance_bin':
            if d['b_type'] is None:
                step = (6-2)/8.0
                start = 2
                b = 9
                for i in range(0, 9):
                    if d['distance'] < (start+i*step):
                        b = i
                        break
                e_t.append(b+5)
            else:
                e_t += [i+1 for i, x in enumerate([rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                   rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC])
                        if x == d['b_type']]
        elif e_representation == 'raw_distance':
            if d['b_type'] is None:
                remove_edges += [(n1, n2)]
            else:
                e_t.append(d['distance'])
                e_t += [int(d['b_type'] == x) for x in [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                        rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]]
        else:
            print('Incorrect Edge representation transform')
            quit()
        if e_t:
            e[(n1, n2)] = e_t
    for edg in remove_edges:
        g.remove_edge(*edg)
    return nx.to_numpy_matrix(g), e
    

def normalize_data(data, mean, std):
    data_norm = (data-mean)/std
    return data_norm


def get_values(obj, start, end, prop):
    vals = []
    for i in range(start, end):
        v = {}
        if 'degrees' in prop:
            v['degrees'] = set(sum(obj[i][0][0].sum(axis=0, dtype='int').tolist(), []))
        if 'edge_labels' in prop:
            v['edge_labels'] = set(sum(list(obj[i][0][2].values()), []))
        if 'target_mean' in prop or 'target_std' in prop:
            v['params'] = obj[i][1]
        vals.append(v)
    return vals


def get_graph_stats(graph_obj_handle, prop='degrees'):
    # if prop == 'degrees':
    num_cores = multiprocessing.cpu_count()
    inputs = [int(i*len(graph_obj_handle)/num_cores) for i in range(num_cores)] + [len(graph_obj_handle)]
    res = Parallel(n_jobs=num_cores)(delayed(get_values)(graph_obj_handle, inputs[i], inputs[i+1], prop) for i in range(num_cores))

    stat_dict = {}

    if 'degrees' in prop:
        stat_dict['degrees'] = list(set([d for core_res in res for file_res in core_res for d in file_res['degrees']]))
    if 'edge_labels' in prop:
        stat_dict['edge_labels'] = list(set([d for core_res in res for file_res in core_res for d in file_res['edge_labels']]))
    if 'target_mean' in prop or 'target_std' in prop:
        param = np.array([file_res['params'] for core_res in res for file_res in core_res])
    if 'target_mean' in prop:
        stat_dict['target_mean'] = np.mean(param, axis=0)
    if 'target_std' in prop:
        stat_dict['target_std'] = np.std(param, axis=0)

    return stat_dict


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    pred = pred.type_as(target)
    target = target.type_as(pred)
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def collate_g(batch):

    batch_sizes = np.max(np.array([[len(input_b[1]), len(input_b[1][0]), len(input_b[2]),
                                len(list(input_b[2].values())[0])]
                                if input_b[2] else
                                [len(input_b[1]), len(input_b[1][0]), 0,0]
                                for (input_b, target_b) in batch]), axis=0)

    g = np.zeros((len(batch), batch_sizes[0], batch_sizes[0]))
    h = np.zeros((len(batch), batch_sizes[0], batch_sizes[1]))
    e = np.zeros((len(batch), batch_sizes[0], batch_sizes[0], batch_sizes[3]))

    target = np.zeros((len(batch), len(batch[0][1])))

    for i in range(len(batch)):

        num_nodes = len(batch[i][0][1])

        # Adjacency matrix
        g[i, 0:num_nodes, 0:num_nodes] = batch[i][0][0]

        # Node features
        h[i, 0:num_nodes, :] = batch[i][0][1]

        # Edges
        for edge in batch[i][0][2].keys():
            e[i, edge[0], edge[1], :] = batch[i][0][2][edge]
            e[i, edge[1], edge[0], :] = batch[i][0][2][edge]

        # Target
        target[i, :] = batch[i][1]

    g = torch.FloatTensor(g)
    h = torch.FloatTensor(h)
    e = torch.FloatTensor(e)
    target = torch.FloatTensor(target)

    return g, h, e, target





In [31]:
class Qm9(data.Dataset):

    # Constructor
    def __init__(self, root_path, ids, vertex_transform=utils.qm9_nodes, edge_transform=utils.qm9_edges,
                 target_transform=None, e_representation='raw_distance'):
        self.root = root_path
        self.ids = ids
        self.vertex_transform = vertex_transform
        self.edge_transform = edge_transform
        self.target_transform = target_transform
        self.e_representation = e_representation

    def __getitem__(self, index):
        g, target = xyz_graph_reader(os.path.join(self.root, self.ids[index]))
        if self.vertex_transform is not None:
            h = self.vertex_transform(g)

        if self.edge_transform is not None:
            g, e = qm9_edges(g, self.e_representation)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (g, h, e), target

    def __len__(self):
        return len(self.ids)

    def set_target_transform(self, target_transform):
        self.target_transform = target_transform

In [32]:
# Parse optios for downloading
parser = argparse.ArgumentParser(description='QM9 Object.')
# Optional argument
parser.add_argument('--root', nargs=1, help='Specify the data directory.', default=['data-parse/chemfindata/'])

args = parser.parse_args([])
root = args.root[0]

files = [f for f in os.listdir(root) if os.path.isfile(os.path.join(root, f))]

idx = np.random.permutation(len(files))
idx = idx.tolist()

valid_ids = [files[i] for i in idx[0:1000]]
test_ids  = [files[i] for i in idx[1000:2000]]
train_ids = [files[i] for i in idx[2000:]]

data_train = Qm9(root, train_ids, )
data_valid = Qm9(root, valid_ids)
data_test = Qm9(root, test_ids)

print(len(data_train))
print(len(data_valid))
print(len(data_test))

print(data_train[0])
print('\n')
print(data_valid[0])
print('\n')
print(data_test[0])
print('\n')

start = time.time()
print(get_graph_stats(data_valid, 'degrees'))
end = time.time()
print('Time Statistics Par')
print(end - start)


1346
1000
1000
((matrix([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.

         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), [[0, 1, 0, 0, 0, 6, -0.48, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 8, -0.68, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 15, 1.47, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 16, -0.57, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 8, -0.57, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 6, -0.55, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 16, -0.55, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 6, -0.42, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 7, 0.09, 0, 1, 0, 0, 1, 0], [0, 1, 0, 0, 0, 6, 0.09, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 8, 0.54, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 6, 0.54, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, 0.53, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, -0.15, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, -0.15, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, -0.15, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, -0.15, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 6, 0.28, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 8, 0.28, 1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0.0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0.0, 0, 0, 0, 0

PicklingError: Could not pickle the task to send it to the workers.

In [6]:
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

# Argument parser
parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='our_data')
parser.add_argument('--datasetPath', default='./data-parse/chemfindata/', help='dataset path')
parser.add_argument('--logPath', default='./log/our_data/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/our_data/mpnn/', help='plot path')
parser.add_argument('--resume', default='./checkpoint/our_data/mpnn/',
                    help='path to latest checkpoint')
# Optimization Options
parser.add_argument('--batch-size', type=int, default=20, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=360, metavar='N',
                    help='Number of epochs to train (default: 360)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=2, help='Pre-fetching threads.')

best_er1 = 0

In [ ]:
data_valid

In [7]:
stat_dict = get_graph_stats(data_valid, ['target_mean', 'target_std'])


In [16]:
args = parser.parse_args([])

In [17]:
print('Define model')
# Select one graph
g_tuple, l = data_train[0]
g, h_t, e = g_tuple

print('\tStatistics')
stat_dict = get_graph_stats(data_valid, ['target_mean', 'target_std'])

data_train.set_target_transform(lambda x: datasets.utils.normalize_data(x,stat_dict['target_mean'],stat_dict['target_std']))
data_valid.set_target_transform(lambda x: datasets.utils.normalize_data(x, stat_dict['target_mean'],stat_dict['target_std']))
data_test.set_target_transform(lambda x: datasets.utils.normalize_data(x, stat_dict['target_mean'], stat_dict['target_std']))

# Data Loader
train_loader = torch.utils.data.DataLoader(data_train,
                                           batch_size=args.batch_size, shuffle=True,
                                           collate_fn=datasets.utils.collate_g,
                                        pin_memory=True)
valid_loader = torch.utils.data.DataLoader(data_valid,
                                           batch_size=args.batch_size, collate_fn=datasets.utils.collate_g,
                                           pin_memory=True)
test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=args.batch_size, collate_fn=datasets.utils.collate_g,
                                          pin_memory=True)

Define model
	Statistics


In [8]:
args.cuda =None# not args.no_cuda and torch.cuda.is_available()

In [9]:
print('\tCreate model')
in_n = [len(h_t[0]), len(list(e.values())[0])]
hidden_state_size = 73
message_size = 73
n_layers = 3
l_target = len(l)
type ='regression'
model = MPNN(in_n, hidden_state_size, message_size, n_layers, l_target, type=type)
del in_n, hidden_state_size, message_size, n_layers, l_target, type

print('Optimizer')
optimizer = optim.Adam(model.parameters(), lr=args.lr)

criterion = nn.MSELoss()

evaluation = lambda output, target: torch.mean(torch.abs(output - target) / torch.abs(target))


lr_step = (args.lr-args.lr*args.lr_decay)/(args.epochs*args.schedule[1] - args.epochs*args.schedule[0])

# get the best checkpoint if available without training
if args.resume:
    checkpoint_dir = args.resume
    best_model_file = os.path.join(checkpoint_dir, 'model_best.pth')
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if os.path.isfile(best_model_file):
        print("=> loading best model '{}'".format(best_model_file))
        checkpoint = torch.load(best_model_file)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_er1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded best model '{}' (epoch {})".format(best_model_file, checkpoint['epoch']))
    else:
        print("=> no best model found at '{}'".format(best_model_file))

print('Check cuda')
if args.cuda:
    print('\t* Cuda')
    model = model.cuda()
    criterion = criterion.cuda()


	Create model
Optimizer
=> loading best model './checkpoint/our_data/mpnn/model_best.pth'
=> loaded best model './checkpoint/our_data/mpnn/model_best.pth' (epoch 11)
Check cuda


In [10]:
model

MPNN(
  (m): ModuleList(
    (0): MessageFunction(
      (learn_args): ParameterList()
      (learn_modules): ModuleList(
        (0): NNet(
          (fcs): ModuleList(
            (0): Linear(in_features=5, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=128, bias=True)
            (3): Linear(in_features=128, out_features=5329, bias=True)
          )
        )
      )
    )
  )
  (u): ModuleList(
    (0): UpdateFunction(
      (learn_args): ParameterList()
      (learn_modules): ModuleList(
        (0): GRU(73, 73)
      )
    )
  )
  (r): ReadoutFunction(
    (learn_args): ParameterList()
    (learn_modules): ModuleList(
      (0): NNet(
        (fcs): ModuleList(
          (0): Linear(in_features=146, out_features=128, bias=True)
          (1): Linear(in_features=128, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=128, bias=True)
          (3):

In [10]:
def train(train_loader, model, criterion, optimizer, epoch, evaluation, logger):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    error_ratio = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (g, h, e, target) in enumerate(train_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Measure data loading time
        data_time.update(time.time() - end)

        optimizer.zero_grad()

        # Compute output
        output = model(g, h, e)
        train_loss = criterion(output, target)

        # Logs
        losses.update(train_loss.item(), g.size(0))
        error_ratio.update(evaluation(output, target).item(), g.size(0))

        # compute gradient and do SGD step
        train_loss.backward()
        optimizer.step()

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:

            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Error Ratio {err.val:.4f} ({err.avg:.4f})'
                  .format(epoch, i, len(train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, err=error_ratio))
                          

    print('Epoch: [{0}] Avg Error Ratio {err.avg:.3f}; Average Loss {loss.avg:.3f}; Avg Time x Batch {b_time.avg:.3f}'
          .format(epoch, err=error_ratio, loss=losses, b_time=batch_time))


def validate(val_loader, model, criterion, evaluation, logger=None):
    batch_time = AverageMeter()
    losses = AverageMeter()
    error_ratio = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (g, h, e, target) in enumerate(val_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output = model(g, h, e)

        # Logs
        losses.update(criterion(output, target).item(), g.size(0))
        error_ratio.update(evaluation(output, target).item(), g.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:
            
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Error Ratio {err.val:.4f} ({err.avg:.4f})'
                  .format(i, len(val_loader), batch_time=batch_time,
                          loss=losses, err=error_ratio))

    print(' * Average Error Ratio {err.avg:.3f}; Average Loss {loss.avg:.3f}'
          .format(err=error_ratio, loss=losses))


    return error_ratio.avg


In [13]:
for epoch in range(0, args.epochs):

    if epoch > args.epochs * args.schedule[0] and epoch < args.epochs * args.schedule[1]:
        args.lr -= lr_step
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, evaluation, logger=None)

    # evaluate on test set
    er1 = validate(valid_loader, model, criterion, evaluation)

    is_best = er1 > best_er1
    best_er1 = min(er1, best_er1)
    utils.save_checkpoint({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_er1': best_er1,
                           'optimizer': optimizer.state_dict(), }, is_best=is_best, directory=args.resume)

    # Logger step
    #logger.log_value('learning_rate', args.lr).step()

# get the best checkpoint and test it with test set
if args.resume:
    checkpoint_dir = args.resume
    best_model_file = os.path.join(checkpoint_dir, 'model_best.pth')
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if os.path.isfile(best_model_file):
        print("=> loading best model '{}'".format(best_model_file))
        checkpoint = torch.load(best_model_file)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_er1']
        model.load_state_dict(checkpoint['state_dict'])
        if args.cuda:
            model.cuda()
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded best model '{}' (epoch {})".format(best_model_file, checkpoint['epoch']))
    else:
        print("=> no best model found at '{}'".format(best_model_file))

# For testing
validate(test_loader, model, criterion, evaluation)

Epoch: [0][20/68]	Time 3.070 (4.912)	Data 0.392 (0.416)	Loss 0.4190 (0.4555)	Error Ratio 0.8111 (2.7604)
Epoch: [0][40/68]	Time 3.489 (4.631)	Data 0.386 (0.410)	Loss 0.2396 (0.6721)	Error Ratio 1.0590 (1.9948)
Epoch: [0][60/68]	Time 5.629 (4.664)	Data 0.422 (0.412)	Loss 0.4082 (0.7152)	Error Ratio 1.0468 (1.7517)
Epoch: [0] Avg Error Ratio 1.710; Average Loss 0.697; Avg Time x Batch 4.574
Test: [20/50]	Time 1.362 (1.827)	Loss 1.5316 (0.7793)	Error Ratio 0.7588 (2.1140)
Test: [40/50]	Time 2.400 (2.243)	Loss 0.4271 (0.8062)	Error Ratio 0.9512 (3.6430)
 * Average Error Ratio 3.224; Average Loss 0.728
Epoch: [1][20/68]	Time 4.745 (4.430)	Data 0.446 (0.401)	Loss 0.2681 (0.5095)	Error Ratio 0.8766 (1.2672)
Epoch: [1][40/68]	Time 5.469 (4.635)	Data 0.442 (0.409)	Loss 0.3070 (0.7446)	Error Ratio 0.8496 (1.3444)
Epoch: [1][60/68]	Time 4.174 (4.897)	Data 0.368 (0.416)	Loss 1.6426 (0.6715)	Error Ratio 1.5743 (1.3916)
Epoch: [1] Avg Error Ratio 1.382; Average Loss 0.660; Avg Time x Batch 4.723
Tes

KeyboardInterrupt: 

In [14]:
validate(test_loader, model, criterion, evaluation)

Test: [20/50]	Time 2.103 (1.969)	Loss 0.1250 (0.6039)	Error Ratio 0.8150 (3.9243)
Test: [40/50]	Time 3.161 (2.227)	Loss 0.4517 (0.7833)	Error Ratio 1.4968 (2.7592)
 * Average Error Ratio 2.513; Average Loss 0.750


2.5131032884120943

In [ ]:
from __future__ import print_function

import networkx as nx
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.cm as cm
import os
import warnings


__author__ = "Pau Riba, Anjan Dutta"
__email__ = "priba@cvc.uab.cat, adutta@cvc.uab.cat"


"""
   Plots a Graph with the library networkx
"""

class Plotter():
    # Constructor
    def __init__(self, plot_dir = './'):
        self.plotdir = plot_dir

        if os.path.isdir(plot_dir):
            # clean previous logged data under the same directory name
            self._remove(plot_dir)

        os.makedirs(plot_dir)


    @staticmethod
    def _remove(path):
        """ param <path> could either be relative or absolute. """
        if os.path.isfile(path):
            os.remove(path)  # remove the file
        elif os.path.isdir(path):
            import shutil
            shutil.rmtree(path)  # remove dir and all contains

    def plot_graph(self, am, position=None, cls=None, fig_name='graph.png'):

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")

            g = nx.from_numpy_matrix(am)

            if position is None:
                position=nx.drawing.circular_layout(g)

            fig = plt.figure()

            if cls is None:
                cls='r'
            else:
                # Make a user-defined colormap.
                cm1 = mcol.LinearSegmentedColormap.from_list("MyCmapName", ["r", "b"])

                # Make a normalizer that will map the time values from
                # [start_time,end_time+1] -> [0,1].
                cnorm = mcol.Normalize(vmin=0, vmax=1)

                # Turn these into an object that can be used to map time values to colors and
                # can be passed to plt.colorbar().
                cpick = cm.ScalarMappable(norm=cnorm, cmap=cm1)
                cpick.set_array([])
                cls = cpick.to_rgba(cls)
                plt.colorbar(cpick, ax=fig.add_subplot(111))


            nx.draw(g, pos=position, node_color=cls, ax=fig.add_subplot(111))

            fig.savefig(os.path.join(self.plotdir, fig_name))


In [ ]:
m=Plotter(plot_dir='/image')

In [15]:
 for i, (g, h, e, target) in enumerate(train_loader):
    g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)


tensor([[ 2.4563, -0.7447],
        [-0.5977, -0.0359],
        [-0.0172, -0.4512],
        [-0.6062, -0.7089],
        [-0.4702,  0.6776],
        [-0.3575, -0.0813]])

In [16]:
output=model(g,h,e)

In [17]:
target[0]

tensor([-0.8727, -0.1155])

In [18]:
output[0]

tensor([-1.1039, -0.0164], grad_fn=<SelectBackward>)

In [19]:
for i in range(len(output[0])):
    print(output[0][i]*stat_dict['target_std'][i]+stat_dict['target_mean'][i])

tensor(-5.8529, grad_fn=<AddBackward0>)
tensor(1.6046, grad_fn=<AddBackward0>)


In [20]:
for i in range(len(target[0])):
    print(target[0][i]*stat_dict['target_std'][i]+stat_dict['target_mean'][i])

tensor(0.5159)
tensor(1.4800)
